### SI 649 Final Project

Map of different programs spending

In [3]:
import pandas as pd
import altair as alt
from vega_datasets import data

# data
df = pd.read_csv("Postsecondary_School_Locations_2023-24.csv", dtype=str)
df = df.dropna(subset=["LAT","LON","nil_value","2024_conference"])
df["LAT"]       = pd.to_numeric(df["LAT"], errors="coerce")
df["LON"]       = pd.to_numeric(df["LON"], errors="coerce")
df["nil_value"] = pd.to_numeric(df["nil_value"], errors="coerce")

# click selection
conf_sel = alt.selection_point(
    fields=["2024_conference"],
    empty="all",
    on="click"
)

# states background
states = alt.topo_feature(data.us_10m.url, "states")
states_layer = alt.Chart(states).mark_geoshape(
    fill="#f5f5f5", stroke="grey", strokeWidth=0.5
)

# map with bubbles
bubbles = (
    alt.Chart(df).mark_circle(opacity=0.8, stroke="black", strokeWidth=0.5).encode(
         longitude="LON:Q",
         latitude="LAT:Q",
         size=alt.Size(
             "nil_value:Q",
             title=None,
             legend=None,
             scale=alt.Scale(range=[100,2000])
         ),color=alt.Color("nil_value:Q", title="NIL Total ($)", scale=alt.Scale(scheme="yellowgreenblue"), legend=alt.Legend(orient="right")),
         tooltip=[
             alt.Tooltip("school:N", title="School"),
             alt.Tooltip("2024_conference:N", title="Conference"),
             alt.Tooltip("nil_value:Q", title="NIL ($)", format=",")
         ]
     )
     .add_params(conf_sel)
)

map_chart = alt.layer(states_layer, bubbles).project("albersUsa").properties(
    width=600, height=400,
    title="NIL Totals by State"
)

# bar chart
bars = (
    alt.Chart(df)
      .transform_filter(conf_sel)
      .transform_aggregate(
          total_nil="sum(nil_value)",
          groupby=["school"]
      )
      .transform_window(
          rank="rank(total_nil)",
          sort=[alt.SortField("total_nil", order="descending")]
      )
      .transform_filter(alt.datum.rank <= 10)
      .mark_bar()
      .encode(
          x=alt.X(
              "school:N",
              sort=alt.EncodingSortField("total_nil", order="descending"),
              title="School",
              axis=alt.Axis(labelAngle=-45, labelFontSize=10)
          ),
          y=alt.Y("total_nil:Q", title="Total NIL ($)"),
          color=alt.Color(
              "total_nil:Q",
              title="NIL Total ($)",
              scale=alt.Scale(scheme="blues"),
              legend=None
          ),
          tooltip=[
              alt.Tooltip("school:N", title="School"),
              alt.Tooltip("total_nil:Q", title="NIL ($)", format=",")
          ]
      )
      .properties(width=300, height=400, title="NIL Totals by Campus")
)

# combined
final_bubble = alt.hconcat(
    map_chart,
    bars
).resolve_legend(color="shared").configure_view(stroke=None)

final_bubble


alt.HConcatChart(...)

### Chloropleth Version

In [ ]:
# loading in data
df = pd.read_csv("Postsecondary_School_Locations_2023-24.csv", dtype=str)
df = df.dropna(subset=["STFIP","nil_value","school"])
df["STFIP"]     = df["STFIP"].astype(int)
df["nil_value"] = pd.to_numeric(df["nil_value"], errors="coerce")

# aggregate by state
state_data = (
    df
    .groupby("STFIP", as_index=False)
    .agg(total_nil=("nil_value","sum"))
)

states = alt.topo_feature(data.us_10m.url, "states")

map_chart = (
    alt.Chart(states).mark_geoshape().transform_lookup(lookup="id",from_=alt.LookupData(state_data, key="STFIP", fields=["total_nil"])).encode(
          color=alt.Color("total_nil:Q", title="Total NIL ($)",
                          scale=alt.Scale(scheme="yellowgreenblue")),
          tooltip=[
              alt.Tooltip("properties.name:N", title="State"),
              alt.Tooltip("total_nil:Q", title="Total NIL ($)", format=",")
          ]
      )
      .project("albersUsa")
      .properties(width=600, height=400, title="Total NIL by State")
)

# bar chart
bars = (
    alt.Chart(df).transform_aggregate(total_nil="sum(nil_value)", groupby=["school"]).transform_window(
          rank="row_number()",
          sort=[alt.SortField("total_nil", order="descending")]
      ).transform_filter("datum.rank <= 10").mark_bar().encode(
          x=alt.X("total_nil:Q", title="Total NIL ($)"),
          y=alt.Y("school:N",
                  sort=alt.EncodingSortField("total_nil", order="descending"),
                  title="School"),
          color=alt.Color("total_nil:Q", scale=alt.Scale(scheme="blues"), legend=None),
          tooltip=[
              alt.Tooltip("school:N", title="School"),
              alt.Tooltip("total_nil:Q", title="NIL ($)", format=",")
          ]
      )
      .properties(width=300, height=400, title="Top 10 Campuses by NIL")
)

# combined
final = alt.hconcat(
    map_chart,
    bars
).resolve_legend(color="shared").configure_view(stroke=None)

final


alt.HConcatChart(...)

# Scatterplot

In [7]:
# scatterplot


df = pd.read_csv("NIL_conference_map.csv", dtype=str)
df = df.dropna(subset=["nil_dollars", "revenue", "conference"])
df["nil_dollars"] = pd.to_numeric(df["nil_dollars"], errors="coerce")
df["revenue"]     = pd.to_numeric(df["revenue"],     errors="coerce")

# colors
confs   = sorted(df["conference"].unique())
palette = ["#FDE725", "#35B779", "#3B528B"][:len(confs)]

# dropdown menu for conference
conf_param = alt.param(
    name   = "selected_conf",
    bind   = alt.binding_select(options=["All"] + confs, name="Conference "),
    value  = "All"
)

# zoom in and out
zoom = alt.selection_interval(bind="scales")

# scatterplot
scatter = (
    alt.Chart(df)
       .mark_circle(size=200, opacity=0.6, stroke="black", strokeWidth=0.5)
       .encode(
           x=alt.X("revenue:Q", title="Revenue (million $)"),
           y=alt.Y("nil_dollars:Q", title="NIL Dollars (million $)"),
           color=alt.Color(
               "conference:N",
               title="Conference",
               scale=alt.Scale(domain=confs, range=palette)
           ),
           tooltip=[
               alt.Tooltip("school:N", title="School"),
               alt.Tooltip("conference:N", title="Conference"),
               alt.Tooltip("nil_dollars:Q", title="NIL ($M)", format=",.1f"),
               alt.Tooltip("revenue:Q", title="Revenue ($M)", format=",.1f")
           ]
       )
       .add_params(conf_param, zoom)
       .transform_filter(
           "selected_conf == 'All' || datum.conference == selected_conf"
       )
       .properties(width=600, height=400, title="NIL Dollars vs Revenue for Top 25 Schools"
        ).configure_title(fontSize=18, anchor="start"))

scatter


alt.Chart(...)

## Better colors 

In [ ]:
# data
df = pd.read_csv("NIL_conference_map.csv", dtype=str)
df = df.dropna(subset=["nil_dollars", "revenue", "conference"])
df["nil_dollars"] = pd.to_numeric(df["nil_dollars"], errors="coerce")
df["revenue"]     = pd.to_numeric(df["revenue"],     errors="coerce")


confs = sorted(df["conference"].unique())

# dropdown
conf_param = alt.param(
    "selected_conf",
    bind=alt.binding_select(options=["All"] + confs, name="Conference "),
    value="All"
)

zoom = alt.selection_interval(bind="scales")

# scatterplot
scatter = (
    alt.Chart(df)
      .mark_circle(size=200, opacity=0.6, stroke=None, strokeWidth=0.5)
      .encode(
          x=alt.X("revenue:Q", title="Revenue (million $)"),
          y=alt.Y("nil_dollars:Q", title="NIL Dollars (million $)"),
          color=alt.Color(
              "conference:N",
              title="Conference",
              scale=alt.Scale(
                  domain=confs,
                  scheme="yellowgreenblue"
              )
          ),
          tooltip=[
              alt.Tooltip("school:N",      title="School"),
              alt.Tooltip("conference:N",  title="Conference"),
              alt.Tooltip("nil_dollars:Q", title="NIL ($M)",    format=",.1f"),
              alt.Tooltip("revenue:Q",     title="Revenue ($M)", format=",.1f")
          ]
      )
      .add_params(conf_param, zoom)
      .transform_filter(
          "selected_conf == 'All' || datum.conference == selected_conf"
      )
      .properties(
          width=600,
          height=400,
          title="NIL Dollars vs Revenue for Top 25 Schools"
      )
      .configure_title(fontSize=18, anchor="start")
)

scatter

alt.Chart(...)

In [5]:
# save chart 1
final_bubble.save('NIL_totals.html')

In [8]:
# save chart 2
scatter.save('scatter_revenue.html')